# HOMEWORK 1

## DATABASE CREATION

In [ ]:
pip install pandas

In [ ]:
pip install numpy

In [ ]:
pip install matplotlib

In [ ]:
pip install openpyxl

In [ ]:
pip install seaborn

In [ ]:
pip install numpy 

In [ ]:
import pandas as pd

In [ ]:
#Monthly_reurn include 278 months and 2051 firms (rows)
monthly_return_index_path = '/Users/camillevermenouze/Documents/SMTS2/SEF/ESG_Scores/DS_RI_T_USD_M.xlsx'  # Replace with your Excel file name
df_monthly_ri = pd.read_excel(monthly_return_index_path)



#Monthly_capitalization include 278 months and 2051 firms 
monthly_capitalization_path = '/Users/camillevermenouze/Documents/SMTS2/SEF/ESG_Scores/DS_MV_USD_M.xlsx'  # Replace with your Excel file name
df_monthly_capitalization = pd.read_excel(monthly_capitalization_path)



#M ountry to region data
region_path = '/Users/camillevermenouze/Documents/SMTS2/SEF/CountriesToRegions.xlsx'  # Replace with your Excel file name
df_region = pd.read_excel(region_path)

In [ ]:
df_region

In [ ]:
#let's  keep only the EUR contries

df_region.drop(index=[0,1,2,3], inplace= True )

df_EUR = df_region[df_region['Unnamed: 2'] == 'EUR']

#Rename the column so we can merge later
df_EUR = df_EUR.rename(columns={'% AMER: Cnada + USA' : 'ISIN_First2'})

print(df_EUR)

In [ ]:
#Now merged with the two dataset 
#First step we need to create a new columns with only the 2 first characters of the columns ISIN

#First with the return data 

df_monthly_ri['ISIN_First2'] = df_monthly_ri ['ISIN'].str[:2]
df_monthly_ri.head()

In [ ]:
#With the capitalization now
df_monthly_capitalization['ISIN_First2'] = df_monthly_capitalization ['ISIN'].str[:2]
df_monthly_capitalization.head()

In [ ]:
#Second step we merge the data set with the region, keeping only the one matching the EUR countries
df_EUR_monthly_capitalization = pd.merge(df_EUR,  df_monthly_capitalization, on='ISIN_First2', how='inner')
df_EUR_monthly_ri = pd.merge(df_EUR,  df_monthly_ri, on='ISIN_First2', how='inner')

In [ ]:
df_EUR_monthly_capitalization.head()

In [ ]:
df_EUR_monthly_ri.head()

In [ ]:
df_EUR_monthly_capitalization.to_csv('/Users/camillevermenouze/Documents/SMTS2/SEF/df_EUR_monthly_capitalization.csv', index=False)
df_EUR_monthly_ri.to_csv('/Users/camillevermenouze/Documents/SMTS2/SEF/df_EUR_monthly_return.csv', index=False)
#We have now 508 firms with the data for 281 months

## IMPORT MONTHLY DATASET

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
urlmri = "https://raw.githubusercontent.com/GiammarcoBozzelli/SEF/main/df_EUR_monthly_return.csv"
dfmri = pd.read_csv(urlmri)

urlmcapitalization = "https://raw.githubusercontent.com/GiammarcoBozzelli/SEF/main/df_EUR_monthly_capitalization.csv"
dfmcapitalization = pd.read_csv(urlmcapitalization)

In [ ]:
dfmri.head()

In [ ]:
dfmcapitalization.head()

## IMPORT YEARLY DATASET

In [ ]:
urlyri = 'https://raw.githubusercontent.com/GiammarcoBozzelli/SEF/main/df_EUR_yearly_return.csv'
dfyri = pd.read_csv(urlyri)

urlycapitalization = 'https://raw.githubusercontent.com/GiammarcoBozzelli/SEF/main/df_EUR_yearly_capitalization.csv'
dfycapitalization = pd.read_csv(urlycapitalization)

In [ ]:
dfyri.head()

In [ ]:
dfycapitalization.head()

## TASK 1

Compute the annualized average RI and annualized volatility for all individual assets over the period 2008–2022. Plot and comment the histogram of these distributions. 

In [ ]:
import seaborn as sns

In [ ]:
import numpy as np

### Annualized average RI

In [ ]:
dfmcapitalization.drop(dfmcapitalization.columns[:4], axis=1, inplace=True)

In [ ]:
dfmcapitalization.set_index('NAME', inplace = True)
dfmcapitalizationpost = dfmcapitalization.loc[:, (dfmcapitalization.columns >= '2008-01-01')]
dfmcapitalizationpost

In [ ]:
dfmcapitalizationT = dfmcapitalizationpost.T
dfmcapitalizationT

In [ ]:
print(dfmcapitalizationT.dtypes)

In [ ]:
dfmreturns = dfmcapitalizationT.pct_change()

In [ ]:
dfmreturns

In [ ]:
dfannualizedreturns = dfmreturns.mean() * 12

In [ ]:
dfannualizedreturns

In [ ]:
dfannualizedreturns.plot()

In [ ]:
sns.histplot(dfannualizedreturns)

### Annualized volatility

In [ ]:
deviations = dfmreturns - dfmreturns.mean()
squared_deviations = deviations**2
mean_squared_deviations = squared_deviations.mean()

dfmvolatility = np.sqrt(mean_squared_deviations)
dfmvolatility

In [ ]:
dfannualizedvolatility = dfmvolatility*(12**0.5)
dfannualizedvolatility

In [ ]:
dfannualizedvolatility.plot()

Compute and comment on the correlation between individual average RIs and volatilities in the cross section.

### Correlation

In [ ]:
df = pd.DataFrame()
df["returns"]=dfannualizedreturns

In [ ]:
df["volatility"]=dfannualizedvolatility

In [ ]:
df.corr

In [ ]:
df["corr"] = 0
for i in df.index:
    df.loc[i, "corr"] = df.loc[i, "returns"].corr(df.loc[i, "volatility"], method="pearson")

In [ ]:
df.loc[i]["corr"]


## TASK 2

Report the following statistics for both portfolios (equally-weighted portfolio & the market-cap weighted portfolio with monthly rebalancing over the period 2008–2022): annualized average return, annualized volatility, Sharpe ratio, minimum return, and maximum return. Plot the time series of cumulated returns for both portfolios. (15 points)

### Equally-weighted portfolio

In [ ]:
#Creating a vector with equal weights for the 508 firms
n_columns = 508

#df_weight = pd.DataFrame([1/n_columns] * n_columns).T
weight = 1/508
weight

In [ ]:
#Equally weighted portfolio (average returns of all firms per each month)
mreturnsEW = (dfmreturns * weight).sum(axis=1)
mreturnsEW

In [ ]:
mreturnsEW = mreturnsEW[mreturnsEW.index < '2008-01-01']
mreturnsEW

In [ ]:
annualizedmreturnsEW = ((1+ mreturnsEW.mean())*12)-1
annualizedmreturnsEW

### Market-cap portfolio

In [ ]:
# Calculate weights for each month based on previous month's capitalization
totalcapprevious = dfmcapitalizationT.shift(0).sum(axis=1)
weights = dfmcapitalizationT.shift(0).div(totalcapprevious, axis=0)

# Calculate the value-weighted portfolio returns
portfolioreturnsVW = (dfmreturns.shift(1) * weights).sum(axis=1)
portfolioreturnsVW


In [ ]:
dfmreturns = dfmcapitalizationT.pct_change()